In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
from sklearn.model_selection import train_test_split
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# read preprocessed data
train = pd.read_csv("./train_sampled_reviews.csv")
val = pd.read_csv("./val_sampled_reviews.csv")
test = pd.read_csv("./test_sampled_reviews.csv")

In [ ]:
train_texts = train['Review'].to_numpy()
train_labels = train['Label'].to_numpy()
val_texts = val['Review'].to_numpy()
val_labels = val['Label'].to_numpy()
test_texts = test['Review'].to_numpy()
test_labels = test['Label'].to_numpy()

In [ ]:
def process_tokens(text):
    """
    function to process tokens, replace any unwanted chars
    """
    preprocessed_text = text.lower().replace(",", "").replace(".", "").replace(":", "").replace(")", "").replace("-", "").replace("(", "")
    preprocessed_text = ''.join([i for i in preprocessed_text if not preprocessed_text.isdigit()])
    return preprocessed_text

'''def preprocessing(data, tokenizer):
    """
    preprocessing data to list of tokens
    """
    nlp = English()
    tokenizer = tokenizer(nlp.vocab)
    preprocessed_data = []
    for sentence in data:
        sentence = process_tokens(sentence)
        tokens = tokenizer(sentence)
        tlist = []
        for token in tokens:
            tlist.append(str(token))
        preprocessed_data.append(tlist)
    return preprocessed_data'''

def preprocessing(data, tokenizer):
    preprocessed_data = []
    for sentence in data:
        sentence = process_tokens(sentence)
        tokens = tokenizer(sentence)
        tlist = []
        for token in tokens:
            tlist.append(str(token))
        preprocessed_data.append(tlist)
    return preprocessed_data

nlp = English()
tokenizer = Tokenizer(nlp.vocab)
train_data = preprocessing(train_texts, tokenizer)
val_data = preprocessing(val_texts, tokenizer)
test_data = preprocessing(test_texts, tokenizer)

## Creating a vectorizer to vectorize text and create matrix of features
## Bag of words technique
class Vectorizer():
    def __init__(self, max_features):
        self.max_features = max_features
        self.vocab_list = None
        self.token_to_index = None

    def fit(self, dataset):
        word_dict = {}
        for sentence in dataset:
            for token in sentence:
                if token not in word_dict:
                    word_dict[token] = 1
                else:
                    word_dict[token] += 1
        word_dict = dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))
        end_to_slice = min(len(word_dict), self.max_features)
        word_dict = dict(itertools.islice(word_dict.items(), end_to_slice))
        self.vocab_list = list(word_dict.keys())
        self.token_to_index = {}
        counter = 0
        for token in self.vocab_list:
            self.token_to_index[token] = counter
            counter += 1


    def transform(self, dataset):
        data_matrix = np.zeros((len(dataset), len(self.vocab_list)))
        for i, sentence in enumerate(dataset):
            for token in sentence:
                if token in self.token_to_index:
                    data_matrix[i, self.token_to_index[token]] += 1
        return data_matrix

## max features - top k words to consider only
max_features = 2000

vectorizer = Vectorizer(max_features=max_features)
vectorizer.fit(train_data)

## Checking if the len of vocab = k
X_train = vectorizer.transform(train_data)
X_val = vectorizer.transform(val_data)
X_test = vectorizer.transform(test_data)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

vocab = vectorizer.vocab_list

In [ ]:
X_train.shape

(24000, 2000)

In [ ]:
y_train[:10]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [ ]:
y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)
y_test = to_categorical(y_test, 2)

X_train = X_train.reshape(-1, 1, X_train.shape[1])
X_val = X_val.reshape(-1, 1, X_val.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

y_train = y_train.reshape(-1, 2)
y_val = y_val.reshape(-1, 2)
y_test = y_test.reshape(-1, 2)

print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')

X_train.shape: (24000, 1, 2000), y_train.shape: (24000, 2)


## LSTM Model

In [ ]:
X_train.dtype

dtype('float64')

In [ ]:
model_lstm = None
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(1, max_features), dropout=0.4))
model_lstm.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model_lstm.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])

lstm_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/lstm_best_model'
lstm_checkpoint = ModelCheckpoint(
    filepath=lstm_checkpoint_path,
    save_best_only=True,  # Only save the best model
    monitor='val_accuracy',  # Monitor validation accuracy
    mode='max',  # Save the model when validation accuracy improves
    verbose=1  # Print messages about the saving process
)

print(model_lstm.summary())
history_lstm = model_lstm.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=20,
          callbacks=[lstm_checkpoint])

print(history_lstm.history.keys())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 64)                528640    
                                                                 
 dense_11 (Dense)            (None, 2)                 130       
                                                                 
Total params: 528770 (2.02 MB)
Trainable params: 528770 (2.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20
93/94 [============================>.] - ETA: 0s - loss: 0.4582 - accuracy: 0.7747
Epoch 1: val_accuracy improved from -inf to 0.87472, saving model to /content/drive/MyDrive/advanced_project/models/lstm_best_model
94/94 [==============================] - 10s 86ms/step - loss: 0.4583 - accuracy: 0.7746 - val_loss: 0.3140 - val_accuracy: 0.8747
Epoch 2/20
94/94 [==============================] -

## BiLSTM Model

In [ ]:
model_bilstm = None
model_bilstm = Sequential()
model_bilstm.add(Bidirectional(LSTM(64, dropout=0.4), input_shape=(1, max_features)))
model_bilstm.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model_bilstm.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])

bilstm_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/bilstm_best_model'
bilstm_checkpoint = ModelCheckpoint(
    filepath=bilstm_checkpoint_path,
    save_best_only=True,  # Only save the best model
    monitor='val_accuracy',  # Monitor validation accuracy
    mode='max',  # Save the model when validation accuracy improves
    verbose=1  # Print messages about the saving process
)

print(model_bilstm.summary())
history_bilstm = model_bilstm.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=20,
          callbacks=[bilstm_checkpoint])
print(history_bilstm.history.keys())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirecti  (None, 128)               1057280   
 onal)                                                           
                                                                 
 dense_12 (Dense)            (None, 2)                 258       
                                                                 
Total params: 1057538 (4.03 MB)
Trainable params: 1057538 (4.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20
94/94 [==============================] - ETA: 0s - loss: 0.4216 - accuracy: 0.8003
Epoch 1: val_accuracy improved from -inf to 0.88694, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_best_model
94/94 [==============================] - 25s 224ms/step - loss: 0.4216 - accuracy: 0.8003 - val_loss: 0.2751 - v

##Eval on Test Set

In [ ]:
# test best models on test set
best_lstm = load_model(lstm_checkpoint_path)
score_lstm, acc_lstm = best_lstm.evaluate(X_test, y_test, verbose=0)
print(f'Test loss for LSTM model: {score_lstm:.3f}')
print(f'Test accuracy for LSTM model: {acc_lstm:.3f}')
print()

best_bilstm = load_model(bilstm_checkpoint_path)
score_bilstm, acc_bilstm = best_bilstm.evaluate(X_test, y_test, verbose=0)
print(f'Test loss for BiLSTM model: {score_bilstm:.3f}')
print(f'Test accuracy for BiLSTM model: {acc_bilstm:.3f}')

Test loss for LSTM model: 0.372
Test accuracy for LSTM model: 0.858

Test loss for BiLSTM model: 0.430
Test accuracy for BiLSTM model: 0.857


##Test on Yelp Review Dataset

In [ ]:
yelp_train = pd.read_csv('yelp_train.csv')
yelp_val = pd.read_csv('yelp_val.csv')
yelp_test = pd.read_csv('yelp_test.csv')

yelp_train.head()

,Unnamed: 0,Label,Review
0,5724,1,i was a feeling a little abandoned in creepy t...
1,22162,1,The food & atmosphere at Sushi Rock is awesome...
2,18039,0,2.5 stars. Did I come on the wrong day? We ca...
3,1987,1,"Food here is great, had the chicken schwarma w..."
4,11590,1,"I was really craving some Asian noodles, so af..."


In [ ]:
yelp_train = yelp_train.drop(['Unnamed: 0'], axis=1)
yelp_val = yelp_val.drop(['Unnamed: 0'], axis=1)
yelp_test = yelp_test.drop(['Unnamed: 0'], axis=1)

yelp_train.head()

,Label,Review
0,1,i was a feeling a little abandoned in creepy t...
1,1,The food & atmosphere at Sushi Rock is awesome...
2,0,2.5 stars. Did I come on the wrong day? We ca...
3,1,"Food here is great, had the chicken schwarma w..."
4,1,"I was really craving some Asian noodles, so af..."


In [ ]:
train_labels_y, train_texts_y = yelp_train.values[:,0], yelp_train.values[:,1]
val_labels_y, val_texts_y = yelp_val.values[:,0], yelp_val.values[:,1]
test_labels_y, test_texts_y = yelp_test.values[:,0], yelp_test.values[:,1]

In [ ]:
nlp = English()
tokenizer = Tokenizer(nlp.vocab)

train_data_y = preprocessing(train_texts_y, tokenizer)
val_data_y = preprocessing(val_texts_y, tokenizer)
test_data_y = preprocessing(test_texts_y, tokenizer)

In [ ]:
## max features - top k words to consider only
max_features = 2000

vectorizer_y = Vectorizer(max_features=max_features)
vectorizer_y.fit(train_data_y)

## Checking if the len of vocab = k
X_train_y = vectorizer_y.transform(train_data_y)
X_val_y = vectorizer_y.transform(val_data_y)
X_test_y = vectorizer_y.transform(test_data_y)

y_train_y = np.array(train_labels_y)
y_val_y = np.array(val_labels_y)
y_test_y = np.array(test_labels_y)

vocab_y = vectorizer_y.vocab_list

In [ ]:
y_train_y = y_train_y.astype('int')
y_val_y = y_val_y.astype('int')
y_test_y = y_test_y.astype('int')

y_train_y = to_categorical(y_train_y, 2)
y_val_y = to_categorical(y_val_y, 2)
y_test_y = to_categorical(y_test_y, 2)

X_train_y = X_train_y.reshape(-1, 1, X_train_y.shape[1])
X_val_y = X_val_y.reshape(-1, 1, X_val_y.shape[1])
X_test_y = X_test_y.reshape(-1, 1, X_test_y.shape[1])

y_train_y = y_train_y.reshape(-1, 2)
y_val_y = y_val_y.reshape(-1, 2)
y_test_y = y_test_y.reshape(-1, 2)

print(f'X_train_y.shape: {X_train_y.shape}, y_train_y.shape: {y_train_y.shape}')

X_train_y.shape: (26600, 1, 2000), y_train_y.shape: (26600, 2)


In [ ]:
best_lstm = load_model(lstm_checkpoint_path)

score_lstm_y, acc_lstm_y = best_lstm.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for LSTM model: {score_lstm_y:.3f}')
print(f'Test accuracy for LSTM model: {acc_lstm_y:.3f}')
print()

Test loss for LSTM model: 1.396
Test accuracy for LSTM model: 0.525



In [ ]:
best_bilstm = load_model(bilstm_checkpoint_path)

score_bilstm_y, acc_bilstm_y = best_bilstm.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for BiLSTM model: {score_bilstm_y:.3f}')
print(f'Test accuracy for BiLSTM model: {acc_bilstm_y:.3f}')
print()

Test loss for BiLSTM model: 1.746
Test accuracy for BiLSTM model: 0.546



In [ ]:
# finetuning LSTM
model_lstm_finetuned = load_model(lstm_checkpoint_path)

optimizer = Adam(learning_rate=0.001)

model_lstm_finetuned.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

lstm_ft_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/lstm_finetuned_best_model'
lstm_ft_checkpoint = ModelCheckpoint(
    filepath=lstm_ft_checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

history_lstm_ft = model_lstm_finetuned.fit(
    X_train_y, y_train_y,
    batch_size=256,
    validation_data=(X_val_y, y_val_y),
    epochs=20,
    callbacks=[lstm_ft_checkpoint]
)

# Print history keys
print(history_lstm_ft.history.keys())

Epoch 1/20
104/104 [==============================] - ETA: 0s - loss: 1.1323 - accuracy: 0.5476
Epoch 1: val_accuracy improved from -inf to 0.57607, saving model to /content/drive/MyDrive/advanced_project/models/lstm_finetuned_best_model
104/104 [==============================] - 12s 90ms/step - loss: 1.1323 - accuracy: 0.5476 - val_loss: 0.7945 - val_accuracy: 0.5761
Epoch 2/20
102/104 [============================>.] - ETA: 0s - loss: 0.8023 - accuracy: 0.5929
Epoch 2: val_accuracy improved from 0.57607 to 0.63344, saving model to /content/drive/MyDrive/advanced_project/models/lstm_finetuned_best_model
104/104 [==============================] - 7s 63ms/step - loss: 0.8014 - accuracy: 0.5930 - val_loss: 0.6660 - val_accuracy: 0.6334
Epoch 3/20
103/104 [============================>.] - ETA: 0s - loss: 0.6924 - accuracy: 0.6327
Epoch 3: val_accuracy improved from 0.63344 to 0.68152, saving model to /content/drive/MyDrive/advanced_project/models/lstm_finetuned_best_model
104/104 [======

In [ ]:
best_lstm_ft = load_model(lstm_ft_checkpoint_path)

score_lstm_ft, acc_lstm_ft = best_lstm_ft.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for finetuned LSTM model: {score_lstm_ft:.3f}')
print(f'Test accuracy for finetuned LSTM model: {acc_lstm_ft:.3f}')
print()

Test loss for finetuned LSTM model: 0.265
Test accuracy for finetuned LSTM model: 0.891



In [ ]:
# finetuning BiLSTM
model_bilstm_finetuned = load_model(bilstm_checkpoint_path)

optimizer = Adam(learning_rate=0.001)

model_bilstm_finetuned.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

bilstm_ft_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/bilstm_finetuned_best_model'
bilstm_ft_checkpoint = ModelCheckpoint(
    filepath=bilstm_ft_checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

history_bilstm_ft = model_bilstm_finetuned.fit(
    X_train_y, y_train_y,
    batch_size=256,
    validation_data=(X_val_y, y_val_y),
    epochs=20,
    callbacks=[bilstm_ft_checkpoint]
)

Epoch 1/20
103/104 [============================>.] - ETA: 0s - loss: 1.3505 - accuracy: 0.5607
Epoch 1: val_accuracy improved from -inf to 0.61572, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_finetuned_best_model
104/104 [==============================] - 24s 184ms/step - loss: 1.3485 - accuracy: 0.5608 - val_loss: 0.8857 - val_accuracy: 0.6157
Epoch 2/20
103/104 [============================>.] - ETA: 0s - loss: 0.8867 - accuracy: 0.6204
Epoch 2: val_accuracy improved from 0.61572 to 0.66766, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_finetuned_best_model
104/104 [==============================] - 18s 173ms/step - loss: 0.8867 - accuracy: 0.6203 - val_loss: 0.6770 - val_accuracy: 0.6677
Epoch 3/20
104/104 [==============================] - ETA: 0s - loss: 0.7249 - accuracy: 0.6584
Epoch 3: val_accuracy improved from 0.66766 to 0.70416, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_finetuned_best_model
104/10

In [ ]:
best_bilstm_ft = load_model(bilstm_ft_checkpoint_path)

score_bilstm_ft, acc_bilstm_ft = best_bilstm_ft.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for finetuned BiLSTM model: {score_bilstm_ft:.3f}')
print(f'Test accuracy for finetuned BiLSTM model: {acc_bilstm_ft:.3f}')
print()

Test loss for finetuned BiLSTM model: 0.251
Test accuracy for finetuned BiLSTM model: 0.898



In [ ]:
# Feature extractor approach
model_lstm_fe = load_model(lstm_checkpoint_path)

model_lstm_fe.layers[-1].trainable = False
initial_layer1_weights_values = model_lstm_fe.layers[-1].get_weights()

optimizer = Adam(learning_rate=0.001)

model_lstm_fe.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

lstm_fe_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/lstm_fe_best_model'
lstm_fe_checkpoint = ModelCheckpoint(
    filepath=lstm_fe_checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

history_lstm_fe = model_lstm_fe.fit(
    X_train_y, y_train_y,
    batch_size=256,
    validation_data=(X_val_y, y_val_y),
    epochs=20,
    callbacks=[lstm_fe_checkpoint]
)

# Print history keys
print(history_lstm_fe.history.keys())

Epoch 1/20
102/104 [============================>.] - ETA: 0s - loss: 1.2008 - accuracy: 0.5482
Epoch 1: val_accuracy improved from -inf to 0.57747, saving model to /content/drive/MyDrive/advanced_project/models/lstm_fe_best_model
104/104 [==============================] - 10s 76ms/step - loss: 1.1959 - accuracy: 0.5487 - val_loss: 0.8302 - val_accuracy: 0.5775
Epoch 2/20
103/104 [============================>.] - ETA: 0s - loss: 0.8523 - accuracy: 0.5987
Epoch 2: val_accuracy improved from 0.57747 to 0.63871, saving model to /content/drive/MyDrive/advanced_project/models/lstm_fe_best_model
104/104 [==============================] - 12s 117ms/step - loss: 0.8520 - accuracy: 0.5988 - val_loss: 0.6836 - val_accuracy: 0.6387
Epoch 3/20
103/104 [============================>.] - ETA: 0s - loss: 0.7302 - accuracy: 0.6339
Epoch 3: val_accuracy improved from 0.63871 to 0.67994, saving model to /content/drive/MyDrive/advanced_project/models/lstm_fe_best_model
104/104 [=========================

In [ ]:
# ensure LSTM layer was frozen
final_layer1_weights_values = model_lstm_fe.layers[-1].get_weights()
np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)
np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)

In [ ]:
best_lstm_fe = load_model(lstm_fe_checkpoint_path)

score_lstm_fe, acc_lstm_fe = best_lstm_fe.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for feature extractor LSTM model: {score_lstm_fe:.3f}')
print(f'Test accuracy for feature extractor LSTM model: {acc_lstm_fe:.3f}')
print()

Test loss for feature extractor LSTM model: 0.279
Test accuracy for feature extractor LSTM model: 0.885



In [ ]:
# Feature extractor approach for BiLSTM
model_bilstm_fe = load_model(bilstm_checkpoint_path)

model_bilstm_fe.layers[-1].trainable = False
initial_layer1_weights_values = model_bilstm_fe.layers[-1].get_weights()

optimizer = Adam(learning_rate=0.001)

model_bilstm_fe.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

bilstm_fe_checkpoint_path = '/content/drive/MyDrive/advanced_project/models/bilstm_fe_best_model'
bilstm_fe_checkpoint = ModelCheckpoint(
    filepath=bilstm_fe_checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

history_bilstm_fe = model_bilstm_fe.fit(
    X_train_y, y_train_y,
    batch_size=256,
    validation_data=(X_val_y, y_val_y),
    epochs=20,
    callbacks=[bilstm_fe_checkpoint]
)

# Print history keys
print(history_bilstm_fe.history.keys())

Epoch 1/20
104/104 [==============================] - ETA: 0s - loss: 1.4339 - accuracy: 0.5610
Epoch 1: val_accuracy improved from -inf to 0.61607, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_fe_best_model
104/104 [==============================] - 29s 244ms/step - loss: 1.4339 - accuracy: 0.5610 - val_loss: 0.9595 - val_accuracy: 0.6161
Epoch 2/20
104/104 [==============================] - ETA: 0s - loss: 0.9933 - accuracy: 0.6203
Epoch 2: val_accuracy improved from 0.61607 to 0.67047, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_fe_best_model
104/104 [==============================] - 19s 181ms/step - loss: 0.9933 - accuracy: 0.6203 - val_loss: 0.7187 - val_accuracy: 0.6705
Epoch 3/20
104/104 [==============================] - ETA: 0s - loss: 0.8020 - accuracy: 0.6584
Epoch 3: val_accuracy improved from 0.67047 to 0.71346, saving model to /content/drive/MyDrive/advanced_project/models/bilstm_fe_best_model
104/104 [==================

In [ ]:
best_bilstm_fe = load_model(bilstm_fe_checkpoint_path)

score_bilstm_fe, acc_bilstm_fe = best_bilstm_fe.evaluate(X_test_y, y_test_y, verbose=0)
print(f'Test loss for feature extractor BiLSTM model: {score_bilstm_fe:.3f}')
print(f'Test accuracy for feature extractor BiLSTM model: {acc_bilstm_fe:.3f}')
print()

Test loss for feature extractor BiLSTM model: 0.267
Test accuracy for feature extractor BiLSTM model: 0.890

